# Libraries and Global Variables

In [28]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

import arcpy

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [29]:
# web app folder
folderWebAppDataFolder = r"..\widgets\Resiliency\data"

In [30]:
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )

print(data_folder)
print(intermediate_folder)
print(results_folder)

e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\data
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\intermediate
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\results


In [31]:
processGdb = os.path.join(intermediate_folder,'process.gdb')
processGdb

'e:\\WebAppBuilderForArcGIS_v2.20\\server\\apps\\15\\_dataprep\\intermediate\\process.gdb'

In [32]:
def delIfExists(obj):
    try:
        if arcpy.Exists(obj):
            arcpy.Delete_management(obj)
            #print ("DELTED: " + obj)
        #else:
            #print ("DOESN'T EXIST: " + obj)
    except:
        #print ("PASSED: " + obj)
        pass

# Download Data

In [33]:
# Read in RTP Projects
#item  = gis.content.get("d1bbd19922db433bb6309841ee4f7536")
#lyrRtpPoints = item.layers[0] # points layer is first layer
#lyrRtpLines  = item.layers[1] # lines layer is second layer

fcRtpLinesIn             = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines')
fcRtpLinesSplitPointsTmp = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines_splitpoints_temp')
fcRtpLinesSegsTmp        = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines_segs_temp')
fcRtpLinesSegs           = os.path.join(intermediate_folder,r'process.gdb\RTP_2023_Draft_lines_segs')

split_length = .125 # miles


In [52]:
dfCategories = pd.DataFrame([
  ["AF", "Agricultural and Farmland"            ],
  ["CH", "Cultural and Historic Resources"      ],
  ["EJ", "Environmental Justice Consideration"  ],
  ["FP", "Floodplains"                          ],
  ["GH", "Geological Hazards"                   ],
  ["HW", "Habitat and Wildlife"                 ],
  ["HM", "Hazardous Materials & Contamination"  ],
  ["HY", "Hydrological"                         ],
  ["OS", "Open Space, Parks, and Recreation"    ],
# ["LU", "Land Use"                             ], # DON'T USE FOR NOW
  ["SS", "Steep Slopes"                         ]
], columns=['CategoryCode','CategoryName'])
display(dfCategories)

# GOOGLE SHEEET
#https://docs.google.com/spreadsheets/d/1cpS4wcMXwt9Yd7qZisvJfZ8GcRLhFpcTRJS_l7CBBY0/edit#gid=0

# quick view layers: https://www.arcgis.com/apps/mapviewer/index.html?layers=74a6478eae424c17b96d1677ee1a4654

dfOpenDataLayers = pd.DataFrame([  
  ["OS", "LOWN", "State and Federal Lands"              , "State and Federal Lands"              , "929fe39efa1d43ac8b151c2898ad3efe", "OWNER!='Private'"       , 1.5],
  ["HW", "ELKS", "Elk"                                  , "Elk Habitat"                          , "224dbcbdca174d3e8648a8b5693fd130", "VALUE=='crucial'"       , 1.5],
  ["HW", "DEER", "Mule Deer"                            , "Mule Deer Habitat"                    , "a4ea235cef824862b560c693b4f7959b", "VALUE=='crucial'"       , 1.5],
  ["HW", "PRNG", "Pronghorn"                            , "Pronghorn Habitat"                    , "1cd6e0315a19473d925427f7cc1d02c1", "VALUE=='crucial'"       , 1.5],
 #["HW", "TURK", "Turkey"                               , ""                                     , "9654e665cccd4884adb6a416df4d3444", ""                       , 1.5], # no turkeys in crucial
  ["HW", "MOOS", "Moose"                                , "Moose Habitat"                        , "e66c061ebd4346e3b16ed8786f5d5d78", "VALUE=='crucial'"       , 1.5],
  ["HW", "BEAR", "Black Bear"                           , "Black Bear Habitat"                   , "6440f19a05c149b783baf1705e5f2a51", "VALUE=='crucial'"       , 1.5],
  ["HW", "HARE", "Snowshoe Hare"                        , "Snowshoe Hare Habitat"                , "d151c91e95c74b329814319f3c20b145", "VALUE=='crucial'"       , 1.5],
  ["HW", "GOAT", "Mountain Goat"                        , "Mountain Goat Habitat"                , "b6b369f865d243399bfc80908426d31b", "VALUE=='crucial'"       , 1.5],
  ["HW", "DGRS", "Dusky Grouse"                         , "Dusky Grouse Habitat"                 , "10382c7f5f384fc692d094fcb65bf251", "VALUE=='crucial'"       , 1.5],
 #["HW", "RGRS", "Ruffed Grouse"                        , ""                                     , "a3d3fe1f13324882aa4b7f67253b1c06", ""                       , 1.5], # no crucial near WF
 #["HW", "HPAR", "Hungarian Partridge"                  , ""                                     , "7a7c3a873f0148ab8a188f743b7acff2", ""                       , 1.5], # no crucial
  ["HW", "CHKR", "Chukar"                               , "Chukar Habitat"                       , "97c6f7c74f704f84b687eb0746cc07ec", "VALUE=='crucial'"       , 1.5],
  ["HW", "BISN", "Bison"                                , "Bison Habitat"                        , "9bed043eb87641db88424d1080d03045", "VALUE=='crucial'"       , 1.5],
 #["HW", "BGHR", "Rocky Mountain Bighorn Sheep"         , ""                                     , "82b1e7ce1ec74920bcd40cd7e729c9b3", ""                       , 1.5], # none near WFRC
 #["HW", "RNPH", "Ringed-Neck Pheasant"                 , ""                                     , "56acb32933f4463bb81cebb834595905", ""                       , 1.5], # none near WF
  ["HW", "CSTG", "Columbian Sharp-Tailed Grouse"        , "Columbian Sharp-Tailed Grouse Habitat", "c8e7ae2533d84a9a98268357dafa15da", "VALUE=='crucial'"       , 1.5],
  ["HW", "CAQU", "California Quail"                     , "California Quail Habitat"             , "d5dae6ecfc8d490b87f1f67fd4d16fd9", "VALUE=='crucial'"       , 1.5],
  ["HW", "CABS", "California Bighorn Sheep"             , "California Bighorn Sheep Habitat"     , "4c11e02a83f148dcafacbe203efc7432", ""                       , 1.5], # include all since value field looks mis-coded
  ["HW", "BTPG", "Band-Tailed Pigeon"                   , "Band-Tailed Pigeon Habitat"           , "524a4c4b704244d4979b4d2b56f1d937", "VALUE=='crucial'"       , 1.5],
 #["HW", "GMQU", "Gambel's Quail"                       , ""                                     , "95a538966055498e956817bad0a5835c", ""                       , 1.5], # None near WF
 #["HW", "WTPT", "White-Tailed Ptarmigan"               , ""                                     , "170dbdc20fcc4a138022738167c4a765", ""                       , 1.5], # None near WF
 #["HW", "WWDV", "White-Winged Dove"                    , ""                                     , "a2d03a434d6b4b189784434014fe0eef", ""                       , 1.5], # None near WF
 #["HW", "UDBH", "Utah Desert Bighorn Sheep"            , ""                                     , "8c19124fbce245b5942c9e48c0568fab", ""                       , 1.5], # None near WF
 #["HW", "WTJR", "White-Tailed Jackrabbit"              , ""                                     , "1b02a508fa334d33ba18c0156d027751", ""                       , 1.5], # None near WF
  ["HM", "BFWT", "Brownfields Targeted for Cleanup"     , "Brownfields Targeted for Cleanup"     , "630351c0db444fe7933a08eab40e573e", ""                       , 1.5],
  ["HM", "BFNT", "Brownfields Not Targeted for Cleanup" , "Brownfields Not Targeted for Cleanup" , "5fd65304423d40b38b4d93e50698db70", ""                       , 1.5],
  ["HM", "HZWO", "Hazardous Waste & Used Oil Facilities", "Hazardous Waste & Used Oil Facilities", "8e9e6310622043bca68ffc2298959126", ""                       , 1.5],
 #["HM", "LLWD", "Low-Level Waste & Uranium Mills"      , "Low-Level Waste & Uranium Mills"      , "2c81695cbc644afdba7687e0340b5763", ""                       , 1.5], # DO NOT USE
  ["HM", "SWLN", "Solid Waste Landfills"                , "Solid Waste Landfills"                , "1ce0d598a0e440558c63535661dc2399", ""                       , 1.5],
  ["HM", "FUDS", "Formerly Used Defense Sites"          , "Formerly Used Defense Sites"          , "dbcdc13c51614d109102f1251fe9530b", ""                       , 1.5],
  ["HM", "MMRP", "Military Munitions Response Program"  , "Military Munitions Response Program"  , "c77c53794aeb4c448e1307b31c581909", ""                       , 1.5],
 #["HM", "EWAS", "Enforceable Written Assurances"       , "Enforceable Written Assurances"       , "7299aa0fdc80426ca0e42c42f59e1518", ""                       , 1.5],
  ["HM", "NPRL", "National Priorities List"             , "National Priorities List"             , "f9c01f05050c40d7b86dade7dd877cf9", ""                       , 1.5],
  ["HM", "SFST", "Superfunded Sites"                    , "Superfunded Sites"                    , "929a2e7a8d5347b8b3248a450cc42e33", ""                       , 1.5],
  ["HM", "ICBZ", "Institutional Control Buffer Zones"   , "Institutional Control Buffer Zones"   , "d8ea2ca541724277b01726d98197da32", ""                       , 1.5],
 #["HM", "UTRI", "Utah Toxic Release Inventory Tier II" , "Utah Toxic Release Inventory Tier II" , "2f725ff758e24dc496b17f87b7301ed8", ""                       , 1.5],
  ["HM", "TXRI", "Toxic Release Inventory"              , "Toxic Release Inventory"              , "132ac94ba71d4e9b9bcffa57b07feb54", ""                       , 1.5],
  ["HM", "VCUP", "Voluntary Cleanup Program"            , "Voluntary Cleanup Program"            , "e0316bc7899d4e4896e88fe3a46aafa8", ""                       , 1.5],
 #["HM", "PSTF", "Petroleum Storage Tank Facilities"    , "Petroleum Storage Tank Facilities"    , "4d31fef4543f449ebe04a84909a11023", ""                       , 1.5],
  ["CH", "UHDS", "Utah Historic Districts"              , "Utah Historic Districts"              , "eecae7fdf65f47aeae12ba817aa0d66b", ""                       , 1.5],
  ["EJ", "EQFA", "Equity Focus Areas"                   , "Equity Focus Areas"                   , "2ed6d9b418d942b79323e6be7209d63c", ""                       , 1.5],
  ["OS", "LCPK", "Local Parks"                          , "Local Parks"                          , "818536733987486b8082e4c5dec0a1dd", ""                       , 1.5],
  ["OS", "PAPR", "Public Access Properties (WMA)"       , "Public Access Properties (WMA)"       , "56aadb826ef3442db805553caf8126c0", ""                       , 1.5],
  ["AF", "AGFL", "Agricultural and Farmland"            , "Agricultural and Farmland"            , "e3e7fc9316bb4ad09474401ff46e734f", "Landuse=='Agricultural'", 1.5],
  ["FP", "FLDP", "Floodplains"                          , "Floodplains"                          , "9100130e33034813833a938069b7bf79", ""                       , 1.5],
  ["GH", "QUFL", "Quaternary Faults"                    , "Quaternary Faults"                    , "9c85978d0fb54570bc60bec467e2fa7f", ""                       , 1.5],
  ["SS", "STSL", "Steep Slopes"                         , "Steep Slopes"                         , "d42f60e142dd4366a1c24aae4993f8e3", ""                       , 1.5],
  ["HY", "RIPA", "Riparians"                            , "Riparians"                            , "74a6478eae424c17b96d1677ee1a4654", ""                       , 1.5],
  ["HY", "WETL", "Wetlands"                             , "Wetlands"                             , "dbc6a99d04944dab9f23bb6286843a1d", ""                       , 1.5]
],columns=['CategoryCode','LayerCode','ListName','LayerName','OpenDataKey','QueryFilter','SearchBuffer'])
display(dfOpenDataLayers)

# example: https://opendata.arcgis.com/datasets/a37e501176dd400c9c7498c29e766ba2_0.gdb

#item  = gis.content.get("630351c0db444fe7933a08eab40e573e")
#layer = item.layers[0]

#dfDownloadedLayers = pd.DataFrame([
#  [],
#  [],
#],columns=['LayerName','Filename'])

,CategoryCode,CategoryName
0,AF,Agricultural and Farmland
1,CH,Cultural and Historic Resources
2,EJ,Environmental Justice Consideration
3,FP,Floodplains
4,GH,Geological Hazards
5,HW,Habitat and Wildlife
6,HM,Hazardous Materials & Contamination
7,HY,Hydrological
8,OS,"Open Space, Parks, and Recreation"
9,SS,Steep Slopes


,CategoryCode,LayerCode,ListName,LayerName,OpenDataKey,QueryFilter,SearchBuffer
0,OS,LOWN,State and Federal Lands,State and Federal Lands,929fe39efa1d43ac8b151c2898ad3efe,OWNER!='Private',1.5
1,HW,ELKS,Elk,Elk Habitat,224dbcbdca174d3e8648a8b5693fd130,VALUE=='crucial',1.5
2,HW,DEER,Mule Deer,Mule Deer Habitat,a4ea235cef824862b560c693b4f7959b,VALUE=='crucial',1.5
3,HW,PRNG,Pronghorn,Pronghorn Habitat,1cd6e0315a19473d925427f7cc1d02c1,VALUE=='crucial',1.5
4,HW,MOOS,Moose,Moose Habitat,e66c061ebd4346e3b16ed8786f5d5d78,VALUE=='crucial',1.5
5,HW,BEAR,Black Bear,Black Bear Habitat,6440f19a05c149b783baf1705e5f2a51,VALUE=='crucial',1.5
6,HW,HARE,Snowshoe Hare,Snowshoe Hare Habitat,d151c91e95c74b329814319f3c20b145,VALUE=='crucial',1.5
7,HW,GOAT,Mountain Goat,Mountain Goat Habitat,b6b369f865d243399bfc80908426d31b,VALUE=='crucial',1.5
8,HW,DGRS,Dusky Grouse,Dusky Grouse Habitat,10382c7f5f384fc692d094fcb65bf251,VALUE=='crucial',1.5
9,HW,CHKR,Chukar,Chukar Habitat,97c6f7c74f704f84b687eb0746cc07ec,VALUE=='crucial',1.5


In [53]:
#Check for duplicate codes:
dfOpenDataLayers['LayerCode'].duplicated().any()

False

In [54]:
#item1  = gis.content.get("a4ea235cef824862b560c693b4f7959b")
#item2  = gis.content.get("dbc6a99d04944dab9f23bb6286843a1d")
#flayer1 = item1.layers[0]
#flayer2 = item2.layers[0]
#sdfTemp1 = pd.DataFrame.spatial.from_layer(flayer1)
#sdfTemp2 = pd.DataFrame.spatial.from_layer(flayer1)
#
#sdfTemp1.query("VALUE=='crucial'")


# Split RTP polyline projects into predefined lengths

In [55]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map1 = gis.map('Salt Lake')
#
#sdf = pd.DataFrame.spatial.from_featureclass(fcRtpLinesIn)
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdf.spatial.plot(map_widget = map1)
#
##show map1
#map1

In [56]:
from IPython.display import clear_output, display

# A TOGGLE TO SET SO THAT RUNNING NOTEBOOK AGAIN THIS PROCESS ISN'T REPEATED IF NOT NEEDED, ie if this is set to 'False' don't run
bSplitSegs = False

if bSplitSegs:

    delIfExists(fcRtpLinesSegs)

    bFirst = True

    # Create a search cursor for the feature class
    fields = ["OID@", "SHAPE@", "gis_id"]
    cursor = arcpy.da.SearchCursor(fcRtpLinesIn, fields)

    # Get the total number of rows in the feature class
    count = arcpy.GetCount_management(fcRtpLinesIn)[0]

    # need to loop through each polyline separately since there are overlapping projects and have to be treated separately
    # Loop through each polyline in the input feature class
    for i, row in enumerate(cursor):

        # Get the OID and shape of the current polyline
        oid      = row[0]
        polyline = row[1]
        gid      = row[2]
        
        # delete temp layers
        delIfExists(fcRtpLinesSplitPointsTmp)
        delIfExists(fcRtpLinesSegsTmp)


        # Generate points along the polyline at the specified interval
        arcpy.GeneratePointsAlongLines_management(polyline, fcRtpLinesSplitPointsTmp, 'DISTANCE', Distance = str(split_length) + ' miles')
        
        # Split the polyline at the generated points
        arcpy.SplitLineAtPoint_management(polyline, fcRtpLinesSplitPointsTmp, fcRtpLinesSegsTmp, '5 feet')

        
        # Add a new field to the output feature class to store the original OID of the polyline
        arcpy.AddField_management(fcRtpLinesSegsTmp, "gis_id", "TEXT")

        # Populate the ORIG_OID field with the OID of the original polyline
        with arcpy.da.UpdateCursor(fcRtpLinesSegsTmp, ["gis_id"]) as update_cursor:
            for update_row in update_cursor:
                update_row[0] = gid
                update_cursor.updateRow(update_row)

        if bFirst:
            arcpy.management.Copy(fcRtpLinesSegsTmp, fcRtpLinesSegs)
            bFirst = False
        else:
            arcpy.management.Append(fcRtpLinesSegsTmp, fcRtpLinesSegs)

        # Update the progressor
        progress = "Processing polylines. Done processing: " + str(i+1) + " of " + str(count)
        display(progress, display_id="progress")

        # Clear the previous progress message
        clear_output(wait=True)

    # Clean up the cursor and progressor
    del cursor

took 49 minutes to run

In [57]:
# delete temp layers
delIfExists(fcRtpLinesSplitPointsTmp)
delIfExists(fcRtpLinesSegsTmp)

In [58]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map2 = gis.map('Salt Lake')
#
#sdfSegs = pd.DataFrame.spatial.from_featureclass(fcRtpLinesSegs)
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdfSegs.spatial.plot(map_widget = map2)
#
##show map1
#map2

In [59]:
bProcess         = True  # toggle for entire cell... Set to false if you do not want to download and do not want to rejoin to projects
bReDownloadAll   = True  # Set to True if you want to redownload data... otherwise set to False, this will download data and store in intermediate step... makes joining faster
bReJoinAll       = True  # Set to True if you want to rejoin the projects to the open data layers

gis = GIS(expiration=9999) # set expiration higher so it doesn't time out with the wetlands

if bProcess: 

    import time

    start_time = time.time()

    for index, row in dfOpenDataLayers.iterrows():

        # display all the variables used
        print (row['LayerCode'] + ' -  ' + row['OpenDataKey'] + ' - ' + str(row['SearchBuffer']) + ' -  ' + row['QueryFilter'])

        #try:
        _fc_download = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_orig_fltr')
        _fc_join     = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_joined'   )

        # check to see if replace all toggle is on OR if the feature class already exists in the gdb
        if bReDownloadAll or not arcpy.Exists(_fc_download):

            # delete feature class if it exists
            delIfExists(_fc_download)

            # get data from web
            print ("downloading...")
            _item  = gis.content.get(row['OpenDataKey'])
            _flayer = _item.layers[0]
            _sdfLyr = pd.DataFrame.spatial.from_layer(_flayer)

            # filter data based on query
            print ("filtering...")
            if bool(row['QueryFilter']):
                _sdfLyr = _sdfLyr.query(row['QueryFilter'])

            # if sdf is not empty
            if not _sdfLyr.empty:
                print ("exporting intermediate layer...")
                _sdfLyr.spatial.to_featureclass(_fc_download)

        if bReJoinAll or not arcpy.Exists(_fc_join):

            # only join again if download has been created
            if arcpy.Exists(_fc_download):

                # delete feature class if it exists
                delIfExists(_fc_join)

                # get closest distance
                print ("joining...")
                arcpy.analysis.SpatialJoin(fcRtpLinesSegs, _fc_download, _fc_join, "JOIN_ONE_TO_ONE", "KEEP_COMMON", match_option="CLOSEST", search_radius=str(row['SearchBuffer']) + " mile", distance_field_name="meters" + row['LayerCode'])
                del _sdfLyr
            else:
                print('no features to join to')
                
        end_time = time.time()

        hours, rem = divmod(end_time - start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"done... elapsed time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds") 
        #except:
        #    print ("ERROR: " + row['LayerCode'])

print ('ALL DONE!')

LOWN -  929fe39efa1d43ac8b151c2898ad3efe - 1.5 -  OWNER!='Private'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 0 minutes, 51 seconds
ELKS -  224dbcbdca174d3e8648a8b5693fd130 - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 6 seconds
DEER -  a4ea235cef824862b560c693b4f7959b - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 19 seconds
PRNG -  1cd6e0315a19473d925427f7cc1d02c1 - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 26 seconds
MOOS -  e66c061ebd4346e3b16ed8786f5d5d78 - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 32 seconds
BEAR -  6440f19a05c149b783baf1705e5f2a51 - 1.5

Exception: Could not load the dataset: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
_sdfLyr


,ADMIN,AGENCY,COUNTY,DESIG,Edit_Date,GIS_Acres,GlobalID,Label_Federal,Label_State,NATL_LGD,OBJECTID,OWNER,SHAPE,STATE_LGD,Tribe,UT_LGD
0,BLM,BLM,JUAB,Bankhead Jones,2008-10-01,40.266972,{0C056FD3-6AE7-4F7B-B9BE-72B6F44E943C},None,None,Bankhead-Jones Land Use Lands,1642033,Federal,"{""rings"": [[[-12501076.493700001, 4843012.1837...",Bankhead-Jones Land Use Lands,None,Bankhead-Jones Land Use Lands
1,BLM,BLM,JUAB,Bankhead Jones,2008-10-01,321.951508,{71637F01-A7EB-4D17-8AEB-DF8E95C8D56B},None,None,Bankhead-Jones Land Use Lands,1642034,Federal,"{""rings"": [[[-12493700.2248, 4843457.383699998...",Bankhead-Jones Land Use Lands,None,Bankhead-Jones Land Use Lands
2,BLM,BLM,JUAB,Bankhead Jones,2008-10-01,39.764534,{DBE96387-BD12-411A-BCA1-7240E6846B01},None,None,Bankhead-Jones Land Use Lands,1642035,Federal,"{""rings"": [[[-12501594.2588, 4844587.417300001...",Bankhead-Jones Land Use Lands,None,Bankhead-Jones Land Use Lands
3,BLM,BLM,JUAB,Bankhead Jones,2008-10-01,39.722595,{9F0CF1E1-1DF7-469C-A093-3F922A29EC24},None,None,Bankhead-Jones Land Use Lands,1642036,Federal,"{""rings"": [[[-12486863.9001, 4844563.716600001...",Bankhead-Jones Land Use Lands,None,Bankhead-Jones Land Use Lands
4,BLM,BLM,JUAB,Bankhead Jones,2008-10-01,160.450592,{32D4754C-23B2-4608-81B1-8DFA282BAAEC},None,None,Bankhead-Jones Land Use Lands,1642037,Federal,"{""rings"": [[[-12500549.4019, 4846706.8706], [-...",Bankhead-Jones Land Use Lands,None,Bankhead-Jones Land Use Lands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15099,Tribal,Tribal,DUCHESNE,Indian Reservation,2020-08-20,1168.584717,{BDD0A3D0-C7F4-4873-A93F-4D60AB74F3D8},Uintah and Ouray Tribal Lands,None,Indian Reservation (IR),1657132,Tribal,"{""rings"": [[[-12300712.464499999, 4876910.2003...",Tribal Lands,Ute,Indian Reservation (IR)
15100,Tribal,Tribal,GRAND,Indian Reservation,2021-05-05,0.000248,{11CC6E71-36BA-440B-8C78-73F35D0AC4E8},Uintah and Ouray Tribal Lands,None,Indian Reservation (IR),1657133,Tribal,"{""rings"": [[[-12247485.5887, 4777728.635799997...",Tribal Lands,Ute,Indian Reservation (IR)
15101,Tribal,Tribal,EMERY,Indian Reservation,2021-05-05,1.586535,{1B5D8CE9-9563-4E07-A8E7-23C0862E222F},Uintah and Ouray Tribal Lands,None,Indian Reservation (IR),1657134,Tribal,"{""rings"": [[[-12247485.5999, 4777741.965000004...",Tribal Lands,Ute,Indian Reservation (IR)
15102,BLM,BLM,GRAND,N/A,2008-10-01,0.004566,{2898B04E-1E01-4D3D-B781-0814C1D7AF1F},None,None,Bureau of Land Management (BLM),1657135,Federal,"{""rings"": [[[-12252800.865699999, 4674097.6569...",Bureau of Land Management,None,Bureau of Land Management (BLM)


In [ ]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map3 = gis.map('Salt Lake')
#
#sdf = pd.DataFrame.spatial.from_featureclass(os.path.join(intermediate_folder,'process.gdb\\WETL_joined'))
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdf.spatial.plot(map_widget = map3)
#
##show map1
#map3

In [ ]:
dfD = pd.DataFrame()

for index, row in dfOpenDataLayers.iterrows():
    #try:
    print(row['LayerCode'])
    _fc = os.path.join(processGdb,row['LayerCode'] + '_joined')
    if arcpy.Exists(_fc):
        #display(_fc)

        _sdf = pd.DataFrame.spatial.from_featureclass(_fc)
        
        df = _sdf[['gis_id',"meters" + row['LayerCode'],'ORIG_SEQ']].copy()
        df['LayerCode'] = row['LayerCode']

        df.columns=(['gis_id','dist_meters','ORIG_SEQ','LayerCode'])

        #display(_sdf[['gis_id',"meters" + row['LayerCode']]])

        dfD = pd.concat([dfD,df])

dfD['dist_feet'] = dfD['dist_meters'] / 3.28084

dfD = dfD.drop(columns=('dist_meters'))

dfD

ELKS
DEER
PRNG
MOOS
BEAR
HARE
GOAT
DGRS
CHKR
BISN
CSTG
CAQU
CABS
BTPG
BFWT
BFNT
HZWO
SWLN
FUDS
MMRP
NPRL
SFST
ICBZ
TXRI
VCUP
UHDS
EQFA
LCPK
LOWN
PAPR
WETL
RIPA
AGFL
FLDP
QUFL
STSL


,gis_id,ORIG_SEQ,LayerCode,dist_feet
0,A-B-1,1,ELKS,518.894131
1,A-B-1,2,ELKS,568.088556
2,A-B-1,5,ELKS,715.489381
3,A-B-1,4,ELKS,662.154990
4,A-B-1,3,ELKS,609.157270
...,...,...,...,...
4464,729,21,RIPA,508.894502
4465,729,19,RIPA,624.271305
4466,729,20,RIPA,563.818440
4467,729,22,RIPA,507.683681


In [ ]:
# DON'T USE NONE OR NEW
dfDistance = dfD[['gis_id','dist_feet','ORIG_SEQ','LayerCode']].copy()
dfDistance.columns =['g','ft','s','l']
dfDistance['id'] = dfDistance['g'].astype(str) + '_' + dfDistance['s'].astype(str)
dfDistance = dfDistance[['l','id','ft']]
dfDistance = dfDistance.groupby(['l','id'],as_index=False).agg(ft=('ft','mean'))
dfDistance['ft'] = dfDistance['ft'].astype(int)
dfDistance

,l,id,ft
0,BEAR,11_64,694
1,BEAR,11_65,633
2,BEAR,11_66,572
3,BEAR,11_67,527
4,BEAR,128_18,682
...,...,...,...
251596,WETL,WestWeber_86,0
251597,WETL,WestWeber_87,0
251598,WETL,WestWeber_88,0
251599,WETL,WestWeber_89,0


In [ ]:
dfDistance_pivot = dfD.pivot_table(index=['gis_id','ORIG_SEQ'],columns='LayerCode',aggfunc='mean')
dfDistance_pivot.fillna(-1,inplace=True)
dfDistance_pivot = dfDistance_pivot.astype(int)
dfDistance_pivot.columns = dfDistance_pivot.columns.droplevel(0)
dfDistance_pivot.reset_index(inplace=True)
dfDistance_pivot

LayerCode,gis_id,ORIG_SEQ,BEAR,BFNT,BFWT,BTPG,CAQU,CHKR,DEER,DGRS,...,MOOS,NPRL,PAPR,RIPA,SFST,SWLN,TXRI,UHDS,VCUP,WETL
0,10,1,-1,311,-1,-1,0,-1,-1,-1,...,-1,-1,520,-1,304,-1,-1,-1,-1,53
1,10,2,-1,311,-1,-1,0,-1,-1,-1,...,-1,-1,512,-1,302,-1,-1,-1,-1,50
2,10,3,-1,317,-1,-1,0,-1,-1,-1,...,-1,-1,511,-1,303,-1,-1,-1,-1,57
3,10,4,-1,335,-1,-1,0,-1,-1,-1,...,-1,-1,510,-1,315,-1,-1,-1,-1,90
4,10,5,-1,371,-1,-1,0,-1,-1,-1,...,-1,-1,516,-1,282,-1,-1,-1,-1,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25339,WestWeber,85,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,452,-1,-1,-1,-1,-1,-1,2
25340,WestWeber,86,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,450,-1,-1,-1,-1,-1,-1,0
25341,WestWeber,87,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,460,-1,-1,-1,-1,-1,-1,0
25342,WestWeber,88,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,495,-1,722,-1,-1,-1,-1,0


# Web App Data

In [ ]:
sdfRtpLinesSegs = pd.DataFrame.spatial.from_featureclass(fcRtpLinesSegs)
sdfRtpLinesSegs = sdfRtpLinesSegs.groupby(['gis_id','ORIG_SEQ'],as_index=False).agg(SHAPE=('SHAPE','first'))

#display(sdfRtpLinesSegs)

sdfRtpLinesSegsWithDist = pd.DataFrame.merge(sdfRtpLinesSegs,dfDistance_pivot,on=('gis_id','ORIG_SEQ'),how='left')

# iterate through the list of columns and add them to the DataFrame
for col in dfOpenDataLayers['LayerCode'].unique():
    if col not in sdfRtpLinesSegsWithDist.columns:
        sdfRtpLinesSegsWithDist[col] = None
        
sdfRtpLinesSegsWithDist.fillna(-1,inplace=True)

display(sdfRtpLinesSegsWithDist)

,gis_id,ORIG_SEQ,SHAPE,BEAR,BFNT,BFWT,BTPG,CAQU,CHKR,DEER,...,WETL,PRNG,HARE,BISN,CSTG,CABS,AGFL,FLDP,QUFL,STSL
0,10,1,"{'paths': [[[413426.50009999983, 4507190.0601]...",-1,311,-1,-1,0,-1,-1,...,53,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,10,2,"{'paths': [[[413225.3596999999, 4507186.7063],...",-1,311,-1,-1,0,-1,-1,...,50,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,10,3,"{'paths': [[[413024.19849999994, 4507185.1809]...",-1,317,-1,-1,0,-1,-1,...,57,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,10,4,"{'paths': [[[412823.03369999956, 4507183.9804]...",-1,335,-1,-1,0,-1,-1,...,90,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,10,5,"{'paths': [[[412626.7885999996, 4507150.073899...",-1,371,-1,-1,0,-1,-1,...,101,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25339,WestWeber,85,"{'paths': [[[401672.7982000001, 4566569.049000...",-1,-1,-1,-1,-1,-1,-1,...,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
25340,WestWeber,86,"{'paths': [[[401471.69130000006, 4566573.3893]...",-1,-1,-1,-1,-1,-1,-1,...,0,-1,-1,-1,-1,-1,-1,-1,-1,-1
25341,WestWeber,87,"{'paths': [[[401285.83779999986, 4566643.4121]...",-1,-1,-1,-1,-1,-1,-1,...,0,-1,-1,-1,-1,-1,-1,-1,-1,-1
25342,WestWeber,88,"{'paths': [[[401164.80399999954, 4566800.8816]...",-1,-1,-1,-1,-1,-1,-1,...,0,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
# export segs
bExportSegs = True

if bExportSegs:
    sdfRtpLinesSegsWithDist.spatial.to_featureclass(os.path.join(results_folder,"resiliency_segs.shp"), sanitize_columns=False)
    print("SEGMENTS EXPORTED")
    sdfRtpLinesIn = pd.DataFrame.spatial.from_featureclass(fcRtpLinesIn)[['GIS_ID','SHAPE']]
    sdfRtpLinesIn.spatial.to_featureclass(os.path.join(results_folder,"resiliency_projects_lines.shp"))
    print("PROJECTS EXPORTED")
else:
    print("SEGMENTS NOT EXPORTED")
    print("PROJECTS NOT EXPORTED")


SEGMENTS EXPORTED
PROJECTS EXPORTED


In [ ]:
sdfRtpLinesSegs

,gis_id,ORIG_SEQ,SHAPE
0,10,1,"{""paths"": [[[413426.50009999983, 4507190.0601]..."
1,10,2,"{""paths"": [[[413225.3596999999, 4507186.7063],..."
2,10,3,"{""paths"": [[[413024.19849999994, 4507185.1809]..."
3,10,4,"{""paths"": [[[412823.03369999956, 4507183.9804]..."
4,10,5,"{""paths"": [[[412626.7885999996, 4507150.073899..."
...,...,...,...
25339,WestWeber,85,"{""paths"": [[[401672.7982000001, 4566569.049000..."
25340,WestWeber,86,"{""paths"": [[[401471.69130000006, 4566573.3893]..."
25341,WestWeber,87,"{""paths"": [[[401285.83779999986, 4566643.4121]..."
25342,WestWeber,88,"{""paths"": [[[401164.80399999954, 4566800.8816]..."


In [ ]:
dfDistance_pivot[['gis_id']].drop_duplicates()

LayerCode,gis_id
0,10
17,101
46,103
68,106
82,107
...,...
24938,StGrid
25019,StGrid_2
25125,StGrid_2_BRT
25242,TRAXImp


In [ ]:
# export jsons for categories and layers
dfCategories    .to_json(os.path.join(folderWebAppDataFolder,'cats.json'),orient="table")
dfOpenDataLayers.to_json(os.path.join(folderWebAppDataFolder,'lyrs.json'),orient="table")


# get all gisids
sdfRtpLinesSegs[['gis_id']].drop_duplicates().rename(columns={'gis_id':'g'}).to_json(os.path.join(folderWebAppDataFolder,'gids.json'),index=False,orient="table")

# get only gisid and seq of 
dfDistance_pivot[['gis_id','ORIG_SEQ']].drop_duplicates().rename(columns={'gis_id':'g','ORIG_SEQ':'s'}).to_json(os.path.join(folderWebAppDataFolder,'segs.json'),index=False,orient="table")

In [ ]:
dfDistance.dtypes

l     object
id    object
ft     int32
dtype: object

In [ ]:
# REDO

# FILE BY BUFFER - GIS_ID - CATEGORY
# ROW BY ORIG_SEQ
# RECORDS: 

lstBuffers = [100,200,300,400,500,600]

dfBuffers = dfDistance.copy()

for b in lstBuffers:
    print(b)
    dfBuffers.loc[(dfBuffers['ft']<b),'b' + str(b)] = 1
    dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].fillna(0)
    dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].astype(int)


display(dfBuffers)
dfBuffers.drop(columns=('ft'),inplace=True)

display(dfBuffers)
# export one dist file per gis id

#for index, row in dfDistance[['l']].drop_duplicates().iterrows():
#    print(row['l'])
#    _dfExport = dfDistance[dfDistance['l']==row['l']][['id','d']].set_index('id').to_json(os.path.join(folderWebAppDataFolder + '\\dist' , row['l'] + '.json'),orient='index')

100
200
300
400
500
600


,l,id,ft,b100,b200,b300,b400,b500,b600
0,BEAR,11_64,694,0,0,0,0,0,0
1,BEAR,11_65,633,0,0,0,0,0,0
2,BEAR,11_66,572,0,0,0,0,0,1
3,BEAR,11_67,527,0,0,0,0,0,1
4,BEAR,128_18,682,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
251596,WETL,WestWeber_86,0,1,1,1,1,1,1
251597,WETL,WestWeber_87,0,1,1,1,1,1,1
251598,WETL,WestWeber_88,0,1,1,1,1,1,1
251599,WETL,WestWeber_89,0,1,1,1,1,1,1


,l,id,b100,b200,b300,b400,b500,b600
0,BEAR,11_64,0,0,0,0,0,0
1,BEAR,11_65,0,0,0,0,0,0
2,BEAR,11_66,0,0,0,0,0,1
3,BEAR,11_67,0,0,0,0,0,1
4,BEAR,128_18,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
251596,WETL,WestWeber_86,1,1,1,1,1,1
251597,WETL,WestWeber_87,1,1,1,1,1,1
251598,WETL,WestWeber_88,1,1,1,1,1,1
251599,WETL,WestWeber_89,1,1,1,1,1,1


In [ ]:
dfBuffersWithCat = pd.DataFrame.merge(dfBuffers, dfOpenDataLayers, left_on='l', right_on='LayerCode')
#display(dfBuffersWithCat)

for b in lstBuffers:
    print(b)
    for c in dfCategories['CategoryCode'].unique():
        print(c)
        _lyrs = (dfOpenDataLayers[dfOpenDataLayers['CategoryCode']==c]['LayerCode'].unique())
        _df = dfBuffersWithCat[(dfBuffersWithCat['CategoryCode']==c)][['id','l','CategoryCode','b' + str(b)]]
        _df = _df[_df['b' + str(b)]>0]
        _df.loc[(_df['b' + str(b)]==1),'value'] = _df['l']
        #display(_df)

        _dfp = _df.pivot(index='id', columns='l', values='value')
        _dfp.reset_index(inplace=True)
        _dfp.fillna('',inplace=True)
        _dfp[c] = _dfp[_df['l'].unique()].apply(lambda x: ",".join(filter(None, x)), axis =1)
        _dfp = _dfp[['id',c]]
        _dfp.set_index('id',inplace=True)
        #display(_dfp)

        _dfp.to_json(os.path.join(folderWebAppDataFolder + '\\within_buffers' , 'b' + str(b) + '_' + c + '.json'),orient='index')

100
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
200
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
300
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
400
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
500
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
600
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
